In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define the U-Net model
def unet_model(input_size=(128, 128, 1)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D((2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = Conv2D(2, (3, 3), activation='relu', padding='same')(conv9)
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

# Define the data generators for preprocessing
def create_data_generators(train_images, train_masks, batch_size=32):
    data_gen_args = dict(horizontal_flip=True,
                          vertical_flip=True,
                          rotation_range=90,
                          zoom_range=0.2)

    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed for reproducibility
    seed = 1
    image_generator = image_datagen.flow(train_images, batch_size=batch_size, seed=seed)
    mask_generator = mask_datagen.flow(train_masks, batch_size=batch_size, seed=seed)

    return zip(image_generator, mask_generator)

# Compile and train the model
def train_model(train_images, train_masks, val_images, val_masks, batch_size=32, epochs=50):
    model = unet_model(input_size=(128, 128, 1))

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=[MeanIoU(num_classes=2)])

    train_gen = create_data_generators(train_images, train_masks, batch_size=batch_size)

    history = model.fit(train_gen,
                        validation_data=(val_images, val_masks),
                        steps_per_epoch=len(train_images) // batch_size,
                        epochs=epochs)

    return model, history

# Visualization of results
def visualize_results(model, test_images, test_masks):
    predictions = model.predict(test_images)

    for i in range(5):  # Visualize first 5 test images
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 3, 1)
        plt.title('Input Image')
        plt.imshow(test_images[i].reshape(128, 128), cmap='gray')

        plt.subplot(1, 3, 2)
        plt.title('Ground Truth Mask')
        plt.imshow(test_masks[i].reshape(128, 128), cmap='gray')

        plt.subplot(1, 3, 3)
        plt.title('Predicted Mask')
        plt.imshow(predictions[i].reshape(128, 128), cmap='gray')

        plt.show()

# Load and preprocess your dataset
# Example: Assume train_images, train_masks, val_images, val_masks are numpy arrays
# train_images = ...
# train_masks = ...
# val_images = ...
# val_masks = ...

# Uncomment below lines and use the actual dataset
# model, history = train_model(train_images, train_masks, val_images, val_masks)
# visualize_results(model, val_images, val_masks)


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define the U-Net model
def unet_model(input_size=(128, 128, 1)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D((2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = Conv2D(2, (3, 3), activation='relu', padding='same')(conv9)
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

# Define the data generators for preprocessing
def create_data_generators(train_images, train_masks, batch_size=32):
    data_gen_args = dict(horizontal_flip=True,
                          vertical_flip=True,
                          rotation_range=90,
                          zoom_range=0.2)

    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed for reproducibility
    seed = 1
    image_generator = image_datagen.flow(train_images, batch_size=batch_size, seed=seed)
    mask_generator = mask_datagen.flow(train_masks, batch_size=batch_size, seed=seed)

    return zip(image_generator, mask_generator)

# Compile and train the model
def train_model(train_images, train_masks, val_images, val_masks, batch_size=32, epochs=50):
    model = unet_model(input_size=(128, 128, 1))

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=[MeanIoU(num_classes=2)])

    train_gen = create_data_generators(train_images, train_masks, batch_size=batch_size)

    history = model.fit(train_gen,
                        validation_data=(val_images, val_masks),
                        steps_per_epoch=len(train_images) // batch_size,
                        epochs=epochs)

    return model, history

# Visualization of results
def visualize_results(model, test_images, test_masks):
    predictions = model.predict(test_images)

    for i in range(5):  # Visualize first 5 test images
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 3, 1)
        plt.title('Input Image')
        plt.imshow(test_images[i].reshape(128, 128), cmap='gray')

        plt.subplot(1, 3, 2)
        plt.title('Ground Truth Mask')
        plt.imshow(test_masks[i].reshape(128, 128), cmap='gray')

        plt.subplot(1, 3, 3)
        plt.title('Predicted Mask')
        plt.imshow(predictions[i].reshape(128, 128), cmap='gray')

        plt.show()

# Load and preprocess your dataset
# Example: Assume train_images, train_masks, val_images, val_masks are numpy arrays
# train_images = ...
# train_masks = ...
# val_images = ...
# val_masks = ...

# Uncomment below lines and use the actual dataset
# model, history = train_model(train_images, train_masks, val_images, val_masks)
# visualize_results(model, val_images, val_masks)
